In [2]:
pub type BoxedStm = Box<Stm>;
pub type BoxedExp = Box<Exp>;
pub type BoxedExpList = Box<ExpList>;

#[derive(Debug)]
pub enum Binop {
    Plus,
    Minus,
    Times,
    Div,
}

#[derive(Debug)]
pub enum Stm {
    Compound(BoxedStm, BoxedStm),
    Assign { id: String, exp: BoxedExp },
    Print(BoxedExpList),
}

impl Stm {
    pub fn new_boxed_compound(stm1: BoxedStm, stm2: BoxedStm) -> BoxedStm {
        Box::new(Self::Compound(stm1, stm2))
    }

    pub fn new_boxed_assign(id: String, exp: BoxedExp) -> BoxedStm {
        Box::new(Self::Assign { id, exp })
    }

    pub fn new_boxed_print(exp_list: BoxedExpList) -> BoxedStm {
        Box::new(Self::Print(exp_list))
    }
}

#[derive(Debug)]
pub enum Exp {
    Id(String),
    Num(isize),
    Op {
        left: BoxedExp,
        oper: Binop,
        right: BoxedExp,
    },
    Eseq {
        stm: BoxedStm,
        exp: BoxedExp,
    },
}

impl Exp {
    pub fn new_boxed_id(id: String) -> BoxedExp {
        Box::new(Self::Id(id))
    }

    pub fn new_boxed_num(num: isize) -> BoxedExp {
        Box::new(Self::Num(num))
    }

    pub fn new_boxed_op(left: BoxedExp, oper: Binop, right: BoxedExp) -> BoxedExp {
        Box::new(Self::Op { left, oper, right })
    }

    pub fn new_boxed_eseq(stm: BoxedStm, exp: BoxedExp) -> BoxedExp {
        Box::new(Exp::Eseq { stm, exp })
    }
}

#[derive(Debug)]
pub enum ExpList {
    Pair { head: BoxedExp, tail: BoxedExpList },
    Last(BoxedExp),
}

impl ExpList {
    pub fn new_boxed_pair(head: BoxedExp, tail: BoxedExpList) -> BoxedExpList {
        Box::new(ExpList::Pair { head, tail })
    }

    pub fn new_boxed_last(last: BoxedExp) -> BoxedExpList {
        Box::new(ExpList::Last(last))
    }
}

// a := 5 + 3 ; b := ( print ( a , a - 1) , 10 * a) ; print ( b )
let prog = Stm::new_boxed_compound(
    Stm::new_boxed_assign(
        "a".to_owned(),
        Exp::new_boxed_op(Exp::new_boxed_num(5), Binop::Plus, Exp::new_boxed_num(3)),
    ),
    Stm::new_boxed_compound(
        Stm::new_boxed_assign(
            "b".to_owned(),
            Exp::new_boxed_eseq(
                Stm::new_boxed_print(ExpList::new_boxed_pair(
                    Exp::new_boxed_id("a".to_owned()),
                    ExpList::new_boxed_last(Exp::new_boxed_op(
                        Exp::new_boxed_id("a".to_owned()),
                        Binop::Minus,
                        Exp::new_boxed_num(1),
                    )),
                )),
                Exp::new_boxed_op(
                    Exp::new_boxed_num(10),
                    Binop::Times,
                    Exp::new_boxed_id("a".to_owned()),
                ),
            ),
        ),
        Stm::new_boxed_print(ExpList::new_boxed_last(Exp::new_boxed_id("b".to_owned()))),
    ),
);


（1） 写一个函数 int maxargs(A_stm)，告知给定语句中任意子表达式内的 print 语句的参数个数。例如，maxargs(prog) 的值是 2。

In [3]:
fn maxargs(stms: BoxedStm) -> usize {

    fn exp_maxargs(exp: BoxedExp) -> usize {
        match *exp {
            Exp::Eseq {stm, exp} => maxargs(stm) + exp_maxargs(exp),
            Exp::Op { left, right, .. } => {
                exp_maxargs(left).max(exp_maxargs(right))
            },
            _ => 0
        }
    }

    fn explist_maxargs(exp_list: BoxedExpList) -> usize {

        match *exp_list {
            ExpList::Pair { head, tail} => {
                exp_maxargs(head).max(1 + explist_maxargs(tail))
            },
            ExpList::Last(exp) => 1 + exp_maxargs(exp),
        }
    }

    match *stms {
        Stm::Compound(stm1, stm2) => {
            maxargs(stm1).max(maxargs(stm2))
        },
        Stm::Assign {exp, ..} => exp_maxargs(exp),
        Stm::Print(exp_list) => explist_maxargs(exp_list)
    }
}

maxargs(prog)

2